# Data Loading and Cleaning

In [1]:
import pandas as pd
import data_cleaning as dc

# Load the dataset
df = pd.read_csv('./job_descriptions_20k.csv')
df = df.head(1000)

df['skills'] = df['skills'].apply(dc.preprocess_document)

# Prepare the dataset
X = dc.prepare_dataset(df)

In [2]:
from sklearn.preprocessing import MultiLabelBinarizer

# Initialize the MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['skills'])

## Model Definition

In [3]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MaxAbsScaler

# Initialize the Logistic Regression model
model = make_pipeline(MaxAbsScaler(), OneVsRestClassifier(LogisticRegression(solver='lbfgs')))

# Train the model on the training data
model.fit(X_train, y_train)

/Users/seby/Desktop/School/2024_Spring/cs4120/group_project/JobAssist/myenv/lib/python3.12/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 85 is present in all training examples.
  warnings.warn(
/Users/seby/Desktop/School/2024_Spring/cs4120/group_project/JobAssist/myenv/lib/python3.12/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 91 is present in all training examples.
  warnings.warn(
/Users/seby/Desktop/School/2024_Spring/cs4120/group_project/JobAssist/myenv/lib/python3.12/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 108 is present in all training examples.
  warnings.warn(
/Users/seby/Desktop/School/2024_Spring/cs4120/group_project/JobAssist/myenv/lib/python3.12/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 130 is present in all training examples.
  warnings.warn(
/Users/seby/Desktop/School/2024_Spring/cs4120/group_project/JobAssist/myenv/lib/python3.12/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 

Pipeline(steps=[('maxabsscaler', MaxAbsScaler()),
                ('onevsrestclassifier',
                 OneVsRestClassifier(estimator=LogisticRegression()))])

## K-Fold Validation Training

In [5]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.multiclass import OneVsRestClassifier
# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import MaxAbsScaler
# from sklearn.model_selection import KFold
# from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
# import numpy as np

# # Initialize the Logistic Regression model pipeline
# model = make_pipeline(MaxAbsScaler(), OneVsRestClassifier(LogisticRegression(solver='lbfgs')))

# # Prepare k-fold cross-validation
# kf = KFold(n_splits=5, shuffle=True, random_state=42)

# # Lists to store metrics for each fold
# accuracies = []
# precisions = []
# recalls = []
# f1_scores = []

# # Convert coo matrix to csr matrix for subscripting
# X = X.tocsr()

# for train_index, test_index in kf.split(X):
#     # Split data
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]
    
#     # Train the model on the training data
#     model.fit(X_train, y_train)
    
#     # Predict on the test data
#     y_pred = model.predict(X_test)
    
#     # Calculate metrics for this fold
#     accuracies.append(accuracy_score(y_test, y_pred))
#     precisions.append(precision_score(y_test, y_pred, average='samples'))
#     recalls.append(recall_score(y_test, y_pred, average='samples'))
#     f1_scores.append(f1_score(y_test, y_pred, average='samples'))

# # Calculate average metric across all folds
# print(f"Average Accuracy: {np.mean(accuracies)}")
# print(f"Average Precision: {np.mean(precisions)}")
# print(f"Average Recall: {np.mean(recalls)}")
# print(f"Average F1 Score: {np.mean(f1_scores)}")

/Users/seby/Desktop/School/2024_Spring/cs4120/group_project/JobAssist/myenv/lib/python3.12/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 85 is present in all training examples.
  warnings.warn(
/Users/seby/Desktop/School/2024_Spring/cs4120/group_project/JobAssist/myenv/lib/python3.12/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 91 is present in all training examples.
  warnings.warn(
/Users/seby/Desktop/School/2024_Spring/cs4120/group_project/JobAssist/myenv/lib/python3.12/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 108 is present in all training examples.
  warnings.warn(
/Users/seby/Desktop/School/2024_Spring/cs4120/group_project/JobAssist/myenv/lib/python3.12/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 130 is present in all training examples.
  warnings.warn(
/Users/seby/Desktop/School/2024_Spring/cs4120/group_project/JobAssist/myenv/lib/python3.12/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 

Average Accuracy: 0.7060000000000001
Average Precision: 0.9434333333333333
Average Recall: 0.851995146988549
Average F1 Score: 0.8746663737786321


/Users/seby/Desktop/School/2024_Spring/cs4120/group_project/JobAssist/myenv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Model Evaluation

In [6]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Predict on the test data
y_pred = model.predict(X_test)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='samples')
recall = recall_score(y_test, y_pred, average='samples')
f1 = f1_score(y_test, y_pred, average='samples')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Accuracy: 0.665
Precision: 0.9191666666666667
Recall: 0.8190468473454575
F1 Score: 0.8426315824878077


/Users/seby/Desktop/School/2024_Spring/cs4120/group_project/JobAssist/myenv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [9]:
# Define a sample datapoint
data = {
    'Job Description': 'We are looking for a Data Scientist who is proficient in Python, R, and Machine Learning',
    'Responsibilities': 'Develop predictive models to solve business problems',
    'Qualifications': 'Bachelor\'s Degree in Computer Science or related field',
    'Work Type': 'Full Time',
    'Preference': 'None',
    'Job Title': 'Data Scientist',
    'Role': 'Data Scientist',
    'Company Size': 1000,
    'skills': 'Python, R, Machine Learning'
}
sample_df = pd.DataFrame([data])
X_sample = dc.prepare_dataset(sample_df)

In [10]:
# Predict the skills required for the sample datapoint
y_sample = model.predict(X_sample)

# Convert the predicted skills back to their original labels
predicted_skills = mlb.inverse_transform(y_sample)
print(predicted_skills)

ValueError: X has 19 features, but MaxAbsScaler is expecting 3003 features as input.